In [64]:
import pandas as pd

orderedListOfHiddenVariables = ['R', 'T', 'L']



class factor:
    def __init__(self, path=None, table=None):
        if path:
            self.table = pd.read_csv(path)
        else:
            self.table=table
       
    def __str__(self):
        return self.table.to_string()
    
    def sumOfTable(self):
        return sum(self.table['value'])
    
        
def productFactor(fact1, fact2):
    fact1 = fact1.table.rename(columns={'value': 'factor_left'}, inplace=False)
    fact2 = fact2.table.rename(columns={'value': 'factor_right'}, inplace=False)
    new_factor = fact1.merge(fact2)
    new_values = []
    
    for index, row in new_factor.iterrows():
        new_values.append(row['factor_left']*row['factor_right'])

    new_factor['value'] = new_values
    new_factor=new_factor.drop(columns=['factor_left','factor_right'])

    return factor(table=new_factor)

    

In [65]:
factorList = []
factorList.append(factor(path="./R.txt"))
factorList.append(factor(path="./T.txt"))     
factorList.append(factor(path="./L.txt"))

def containsVariable(var,factor):
    return var in factor.table.columns
    
def printFactorList(l):
    for i in l:
        print(i.table.to_string())
        print("\n")
    print('Done')
    
def joins(factorList,orderedListOfHiddenVariables):
    for var in orderedListOfHiddenVariables:
        
        #partition our factors
        joinList = [x for x in factorList if containsVariable(var,x)]
        factorList = [x for x in factorList if not containsVariable(var,x)]
        
    
        printFactorList(joinList)
        
        currFactor = joinList.pop(0)
        
        while(len(joinList) > 0):
            currFactor = productFactor(currFactor, joinList.pop(0))
            
        #sum out here if var is a hidden variable, not in loop    
        factorList.append(currFactor)
        
    return currFactor
     
df = joins(factorList,orderedListOfHiddenVariables)
print(df.sumOfTable())

   R  value
0  T    0.1
1  F    0.9


   T  R  value
0  T  T    0.8
1  F  T    0.2
2  T  F    0.1
3  F  F    0.9


Done
   L  T  value
0  T  T    0.3
1  F  T    0.7
2  T  F    0.1
3  F  F    0.9


   R  T  value
0  T  T   0.08
1  T  F   0.02
2  F  T   0.09
3  F  F   0.81


Done
   L  T  R  value
0  T  T  T  0.024
1  T  T  F  0.027
2  F  T  T  0.056
3  F  T  F  0.063
4  T  F  T  0.002
5  T  F  F  0.081
6  F  F  T  0.018
7  F  F  F  0.729


Done
1.0


[<__main__.factor object at 0x0000013CE8650FA0>, <__main__.factor object at 0x0000013CE84D3580>, <__main__.factor object at 0x0000013CE86DE350>]
